# Capstone Project: The Battle of Neighborhoods

Oleksandr Tsapin, 30.11.2020

In [1]:
# Import Excel file with postal codes in Kyiv

import pandas as pd
postal_codes = pd.read_excel("kyiv.xlsx")
postal_codes.head()

FileNotFoundError: [Errno 2] No such file or directory: 'kyiv.xlsx'

In [ ]:
# # import geocoder
# import geocoder 

# latitude=[]
# longitude=[]

# # google API doesn't work, I use arcgis API instead.
# for postal_code in postal_codes['Postal_Code']:
#     g = geocoder.arcgis('{}, Kyiv, Ukraine'.format(postal_code))
#     print(postal_code, g.latlng)
#     while (g.latlng is None):
#         g = geocoder.arcgis('{}, Kyiv, Ukraine'.format(postal_code))
#         print(postal_code, g.latlng)
#     latlng = g.latlng
#     latitude.append(latlng[0])
#     longitude.append(latlng[1])

In [ ]:
# Get latitude and longitude by the address

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Yanusha Korchaka 60, Kyiv'
#address = 'Януша Корчака 60, Київ'

geolocator = Nominatim(user_agent="kyiv_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Yanusha Korchaka 60 are {}, {}.'.format(latitude, longitude))

In [ ]:
# # Get latitude and longitude by the address

# !conda install -c conda-forge geopy --yes
# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# address = 'Оболонський район, Київ'

# geolocator = Nominatim(user_agent="kyiv_explorer")
# location = geolocator.geocode(address)
# latitude = location.latitude
# longitude = location.longitude
# print('The geograpical coordinate {} are {}, {}.'.format(address, latitude, longitude))

In [ ]:
postal_codes['Latitude'] = latitude
postal_codes['Longitude'] = longitude
postal_codes

In [ ]:
# Use geopy library to get the latitude and longitude values of Kyiv
address = 'Kyiv, Ukraine'

geolocator = Nominatim(user_agent="kyiv_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kyiv are {}, {}.'.format(latitude, longitude))

In [ ]:
# Install Folium to visualize the map of Kyiv
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# create map of Kyiv using latitude and longitude values
map_kyiv = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, address, postal_code in zip(postal_codes['Latitude'], postal_codes['Longitude'], postal_codes['Address'], postal_codes['Postal_Code']):
    label = '{}, {}'.format(address, "Our Home :)")
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kyiv)  
    
map_kyiv

In [ ]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'VA0PCQ2AWBYEFFN0WFOHL3IIZFVLVHPIZRO3CP4RFX4XTRS2' # your Foursquare ID
CLIENT_SECRET = 'K5IV2CCAJNNGAL5TB1STKHKYSUHROOYAU2Z1FSW30HISUFIF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# First, let's create the GET request URL. Name your URL url.
latitude = postal_codes.loc[0,'Latitude']
longitude = postal_codes.loc[0,'Longitude']
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

In [ ]:
# Send the GET request and examine the resutls

import requests # library to handle requests

results = requests.get(url).json()
results

In [ ]:
# We know that all the information is in the items key. Let's create the function that extracts the category 
# of the venue.

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
# Now we are ready to clean the json and structure it into a pandas dataframe.

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(100)

In [ ]:
# Let's see how many venues were returned by Foursquare.
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [ ]:
# Let's create a function to repeat the same process to all the neighborhoods in boroughs that contain 
# the word Toronto.

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

toronto_venues = getNearbyVenues(names=postal_codes['District'],
                                   latitudes=postal_codes['Latitude'],
                                   longitudes=postal_codes['Longitude']
                                  )

In [ ]:
# Let's check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
# Let's check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighbourhood').count()

In [ ]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
# Let's do preprocessing. A big part of preprocessing is encoding - representing every single piece of data 
# in a way that a computer can understand (the name literally means "convert to computer code").
# In many branches of computer science, especially machine learning and digital circuit design, 
# One-Hot Encoding is widely used. One-hot Encoding is a type of vector representation in which all of the elements 
# in a vector are 0, except for one, which has 1 as its value, where 1 represents a boolean specifying a category 
# of the element.

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(100)

In [ ]:
# let's examine the new dataframe size.
toronto_onehot.shape

In [ ]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

In [ ]:
# Let's confirm the new size
toronto_grouped.shape

In [ ]:
# Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.

import numpy as np # library to handle data in a vectorized manner

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# Install Folium to visualize the map of Kyiv
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# create map of Kyiv using latitude and longitude values
map_kyiv = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, category in zip(toronto_venues['Venue Latitude'], toronto_venues['Venue Longitude'], toronto_venues['Venue Category']):
    label = '{}, {}'.format(category, "Near Home")
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kyiv)  
    
map_kyiv

In [ ]:
# Save the map in html file

kyiv_map.save("kyiv_map.html")